# <span style='font-family:"Times New Roman"'> <span styel=''> **INTERPRETATION**

*Emile Cohen* 
    
*March 2020*

**Goal:** In this Notebook, we gather code allowin to interpret the cancer panels computed in this section.



---

In [3]:
%run -i '../../../utils/setup_environment.ipy'

import warnings
warnings.filterwarnings('ignore')
from cancer_utils import *
from scipy.stats import fisher_exact, ranksums, chi2, norm
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.gridspec as gridspec

data_path = '../../../data/'

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

In [17]:
cancer_type = 'Breast Cancer'
metrics = 'frac_genome_altered'
master = load_clean_up_master(data_path + 'merged_data/master_file.pkl')
# we remover indeterminate and NA tp53_cn_state samples
master_cancer = master[master['Cancer_Type'] == cancer_type][master['tp53_cn_state'] != 'INDETERMINATE'][~master['tp53_cn_state'].isna()]

group_1 = master_cancer[master_cancer['tp53_res_group'] == 'tp53_res'][metrics]
group_2 = master_cancer[master_cancer['tp53_res_group'] == 'no_tp53_res'][metrics]

median_1 = group_1.median()
median_2 = group_2.median()

statistic, p_value = ranksums(group_1.dropna().values,group_2.dropna().values)

print('median tp53_res: ' + str(median_1))
print('median no_tp53_res: ' + str(median_2))
print('statistic, p-value: ' + str(statistic) +','+ str(p_value))

median tp53_res: 0.229
median no_tp53_res: 0.6495
statistic, p-value: -26.043323653020323,1.6012805178148291e-149
